## File Upload Testing in Azure OpenAI (AI Foundry)

### Environment Setup

In [ ]:
# Import required packages
import os
import shutil
import time
from concurrent.futures import ThreadPoolExecutor
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [3]:
# Set file batch configuration
TOTAL_FILES = 200
FILE_SIZE_KB = 5

In [4]:
# Set up environment variables for Azure OpenAI
AOAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")

### Azure OpenAI Client Setup

In [6]:
# Initialise token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

In [7]:
# Initialise Azure OpenAI client
client = AzureOpenAI(  
    azure_endpoint = AOAI_API_BASE,
    azure_ad_token_provider = token_provider,
    api_version = AOAI_API_VERSION,
)

### File Batch Setup

In [ ]:
# Create test files
def create_test_files():
    # Create test files in current directory subdirectory
    test_dir = os.path.join(os.getcwd(), "test_files")
    
    # Create directory if it doesn't exist
    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)
    os.makedirs(test_dir)
    
    files = []
    content_base = "This is test content for vector store batch upload testing. " * (FILE_SIZE_KB * 15)
    
    print(f"Creating test files in...")
    for i in range(TOTAL_FILES):
        file_path = os.path.join(test_dir, f"test_doc_{i:03d}.txt")
        content = f"Document {i+1}\n{content_base}\nEnd of document {i+1}\nTimestamp: {time.time()}"
        
        with open(file_path, 'w') as f:
            f.write(content)
        files.append(file_path)
    
    print(f"Created {TOTAL_FILES} test files")
    return files, test_dir

test_files, temp_dir = create_test_files()

In [17]:
# Create vector store
def create_vector_store():
    vector_store = client.vector_stores.create(
        name=f"Batch Test Vector Store - {int(time.time())}"
    )
    print(f"Created vector store: {vector_store.id}")
    return vector_store.id

vector_store_id = create_vector_store()

Created vector store: vs_uB9u1bj3FblzSNsMxbUKzwEi


### Option 1: Individual Upload of Files to Azure OpenAI

In [15]:
# Upload files to Azure OpenAI individually
def upload_files_individually(file_paths):
    print(f"Uploading {len(file_paths)} files individually to OpenAI...")
    file_ids = []
    start_time = time.time()
    
    for i, file_path in enumerate(file_paths):
        try:
            with open(file_path, 'rb') as f:
                file_obj = client.files.create(file=f, purpose="assistants")
                file_ids.append(file_obj.id)
                print(f"  Uploaded {i+1}/{len(file_paths)}: {file_obj.id}")
        except Exception as e:
            print(f"  Failed to upload file {i+1}: {e}")
    
    duration = time.time() - start_time
    print(f"Individual upload completed: {len(file_ids)} files in {duration:.2f}s")
    print(f"Rate: {len(file_ids)/duration:.2f} files/second")
    return file_ids, duration

individual_file_ids, individual_duration = upload_files_individually(test_files[:50])


Uploading 50 files individually to OpenAI...
  Uploaded 1/50: assistant-31psNk2GF9gGb5CxLLBbGm
  Uploaded 2/50: assistant-CgtUGQ4uQeEdrJmbxDYBcx
  Uploaded 3/50: assistant-DhddtZi9icff3GcDp6BHtk
  Uploaded 4/50: assistant-NAZb2Dq1Qk1pViA1GKop6g
  Uploaded 5/50: assistant-Ryg7X5uNaztaNaB7eMEFju
  Uploaded 6/50: assistant-6mQfZccYc2mBgUKcGX5umu
  Uploaded 7/50: assistant-WfgdaX8Sdghrqqm2qtAjVW
  Uploaded 8/50: assistant-UEsw4UFjz1QTEfGeEtzqXt
  Uploaded 9/50: assistant-HWmwVcrqzQdoZejvJqt8fH
  Uploaded 10/50: assistant-P6JE5oKdhZJEmLh983JPFH
  Uploaded 11/50: assistant-83MtnFYXYnfrJCRJqWmwxH
  Uploaded 12/50: assistant-Md4eBZveZnA7FG27odCtkY
  Uploaded 13/50: assistant-7Fww5FtDayAPJZRUtC1p9c
  Uploaded 14/50: assistant-6mwWuH5TAnL2zWxd1f9Ckg
  Uploaded 15/50: assistant-EZuzoxBihWRmdZqAKQon7X
  Uploaded 16/50: assistant-Pxyjw8QdfeEMAFc4KaRZS9
  Uploaded 17/50: assistant-J7xr4JdNKhaXB7nx1wMN4q
  Uploaded 18/50: assistant-9czfE5aB43wp3apME8B3sP
  Uploaded 19/50: assistant-QMBseBoUQuBUPq7zW9

### Option 2: Individual Upload of Files to Vector Store

In [19]:
# Add files to Vector Store individually
def add_files_to_vector_store_individually(file_ids, vector_store_id):
    print(f"Adding {len(file_ids)} files to vector store individually...")
    start_time = time.time()
    successful = 0
    
    for i, file_id in enumerate(file_ids):
        try:
            result = client.vector_stores.files.create(
                vector_store_id=vector_store_id,
                file_id=file_id
            )
            successful += 1
            print(f"  Added {i+1}/{len(file_ids)}: {result.id}")
        except Exception as e:
            print(f"  Failed to add file {i+1}: {e}")
    
    duration = time.time() - start_time
    print(f"Individual vector store addition: {successful} files in {duration:.2f}s")
    print(f"Rate: {successful/duration:.2f} files/second")
    return successful, duration

individual_vs_count, individual_vs_duration = add_files_to_vector_store_individually(individual_file_ids, vector_store_id)

Adding 50 files to vector store individually...
  Added 1/50: assistant-31psNk2GF9gGb5CxLLBbGm
  Added 2/50: assistant-CgtUGQ4uQeEdrJmbxDYBcx
  Added 3/50: assistant-DhddtZi9icff3GcDp6BHtk
  Added 4/50: assistant-NAZb2Dq1Qk1pViA1GKop6g
  Added 5/50: assistant-Ryg7X5uNaztaNaB7eMEFju
  Added 6/50: assistant-6mQfZccYc2mBgUKcGX5umu
  Added 7/50: assistant-WfgdaX8Sdghrqqm2qtAjVW
  Added 8/50: assistant-UEsw4UFjz1QTEfGeEtzqXt
  Added 9/50: assistant-HWmwVcrqzQdoZejvJqt8fH
  Added 10/50: assistant-P6JE5oKdhZJEmLh983JPFH
  Added 11/50: assistant-83MtnFYXYnfrJCRJqWmwxH
  Added 12/50: assistant-Md4eBZveZnA7FG27odCtkY
  Added 13/50: assistant-7Fww5FtDayAPJZRUtC1p9c
  Added 14/50: assistant-6mwWuH5TAnL2zWxd1f9Ckg
  Added 15/50: assistant-EZuzoxBihWRmdZqAKQon7X
  Added 16/50: assistant-Pxyjw8QdfeEMAFc4KaRZS9
  Added 17/50: assistant-J7xr4JdNKhaXB7nx1wMN4q
  Added 18/50: assistant-9czfE5aB43wp3apME8B3sP
  Added 19/50: assistant-QMBseBoUQuBUPq7zW9nMk5
  Added 20/50: assistant-Kxo4rT75HHHcSSJAFymdkK
 

### Option 3: Batch Upload of Files to Azure OpenAI and its Vector Store

In [20]:
# Upload files to OpenAI and Vector Store
def batch_upload_to_vector_store(file_paths, batch_size=20):
    print(f"Batch uploading {len(file_paths)} files in batches of {batch_size}...")
    all_file_ids = []
    total_upload_time = 0
    
    # Step 1: Upload all files to OpenAI first
    for i in range(0, len(file_paths), batch_size):
        batch_files = file_paths[i:i+batch_size]
        print(f"Uploading batch {i//batch_size + 1}: files {i+1}-{i+len(batch_files)}")
        
        start_time = time.time()
        batch_file_ids = []
        
        for file_path in batch_files:
            try:
                with open(file_path, 'rb') as f:
                    file_obj = client.files.create(file=f, purpose="assistants")
                    batch_file_ids.append(file_obj.id)
            except Exception as e:
                print(f"    Failed to upload file: {e}")
        
        upload_time = time.time() - start_time
        total_upload_time += upload_time
        all_file_ids.extend(batch_file_ids)
        
        print(f"  Uploaded {len(batch_file_ids)} files in {upload_time:.2f}s")
    
    # Step 2: Create vector store batch
    print(f"Creating vector store batch with {len(all_file_ids)} files...")
    start_time = time.time()
    
    try:
        file_batch = client.vector_stores.file_batches.create(
            vector_store_id=vector_store_id,
            file_ids=all_file_ids
        )
        batch_creation_time = time.time() - start_time
        
        print(f"Vector store batch created: {file_batch.id}")
        print(f"Batch creation time: {batch_creation_time:.2f}s")
        print(f"Total time: {total_upload_time + batch_creation_time:.2f}s")
        
        return file_batch, total_upload_time, batch_creation_time, len(all_file_ids)
    
    except Exception as e:
        print(f"Failed to create vector store batch: {e}")
        return None, total_upload_time, 0, len(all_file_ids)

# Test 2: Batch upload next 50 files
batch_result, batch_upload_time, batch_creation_time, batch_file_count = batch_upload_to_vector_store(test_files[50:100], batch_size=20)


Batch uploading 50 files in batches of 20...
Uploading batch 1: files 1-20
  Uploaded 20 files in 31.07s
Uploading batch 2: files 21-40
  Uploaded 20 files in 27.44s
Uploading batch 3: files 41-50
  Uploaded 10 files in 14.94s
Creating vector store batch with 50 files...
Vector store batch created: vsfb_3626cf46a360475392876cd251c7d13e
Batch creation time: 2.10s
Total time: 75.55s


### Option 4: Concurrent Upload Test

In [21]:
# Upload files concurrently
def concurrent_upload_test(file_paths, max_workers=30):
    print(f"Concurrent upload test: {len(file_paths)} files with {max_workers} workers")
    
    def upload_single_file(file_path):
        start_time = time.time()
        try:
            with open(file_path, 'rb') as f:
                file_obj = client.files.create(file=f, purpose="assistants")
            return {
                'success': True,
                'file_id': file_obj.id,
                'duration': time.time() - start_time,
                'error': None
            }
        except Exception as e:
            return {
                'success': False,
                'file_id': None,
                'duration': time.time() - start_time,
                'error': str(e)
            }
    
    start_time = time.time()
    results = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(upload_single_file, fp) for fp in file_paths]
        
        for i, future in enumerate(futures):
            result = future.result()
            results.append(result)
            status = "SUCCESS" if result['success'] else f"FAILED: {result['error'][:30]}"
            print(f"  File {i+1}: {status} ({result['duration']:.2f}s)")
    
    total_time = time.time() - start_time
    successful = [r for r in results if r['success']]
    failed = [r for r in results if not r['success']]
    
    print(f"Concurrent upload results:")
    print(f"  Total files: {len(file_paths)}")
    print(f"  Successful: {len(successful)}")
    print(f"  Failed: {len(failed)}")
    print(f"  Total time: {total_time:.2f}s")
    print(f"  Rate: {len(successful)/total_time:.2f} files/second")
    
    # Count rate limit errors
    rate_limit_errors = sum(1 for r in failed if '429' in str(r['error']))
    print(f"  Rate limit errors (429): {rate_limit_errors}")
    
    return [r['file_id'] for r in successful], len(successful), total_time

concurrent_file_ids, concurrent_success_count, concurrent_duration = concurrent_upload_test(test_files[100:150], max_workers=30)


Concurrent upload test: 50 files with 30 workers
  File 1: SUCCESS (0.90s)
  File 2: SUCCESS (2.39s)
  File 3: SUCCESS (1.56s)
  File 4: SUCCESS (2.15s)
  File 5: SUCCESS (3.26s)
  File 6: SUCCESS (3.21s)
  File 7: SUCCESS (2.55s)
  File 8: SUCCESS (1.78s)
  File 9: SUCCESS (1.74s)
  File 10: SUCCESS (2.64s)
  File 11: SUCCESS (2.12s)
  File 12: SUCCESS (1.59s)
  File 13: SUCCESS (1.74s)
  File 14: SUCCESS (2.05s)
  File 15: SUCCESS (1.90s)
  File 16: SUCCESS (2.21s)
  File 17: SUCCESS (2.18s)
  File 18: SUCCESS (2.04s)
  File 19: SUCCESS (1.95s)
  File 20: SUCCESS (1.23s)
  File 21: SUCCESS (2.54s)
  File 22: SUCCESS (1.34s)
  File 23: SUCCESS (2.99s)
  File 24: SUCCESS (1.64s)
  File 25: SUCCESS (3.11s)
  File 26: SUCCESS (3.72s)
  File 27: SUCCESS (3.55s)
  File 28: SUCCESS (3.23s)
  File 29: SUCCESS (1.61s)
  File 30: SUCCESS (1.58s)
  File 31: SUCCESS (2.97s)
  File 32: SUCCESS (4.06s)
  File 33: SUCCESS (1.36s)
  File 34: SUCCESS (1.62s)
  File 35: SUCCESS (1.58s)
  File 36: SUCC

### High Concurrency Stress Test

In [22]:
# Perform high concurrency stress test
print("High concurrency stress test (50 files, 50 workers)")
stress_file_ids, stress_success_count, stress_duration = concurrent_upload_test(test_files[150:200], max_workers=50)

High concurrency stress test (50 files, 50 workers)
Concurrent upload test: 50 files with 50 workers
  File 1: SUCCESS (5.59s)
  File 2: SUCCESS (2.61s)
  File 3: SUCCESS (1.49s)
  File 4: SUCCESS (5.08s)
  File 5: SUCCESS (3.04s)
  File 6: SUCCESS (1.50s)
  File 7: SUCCESS (1.97s)
  File 8: SUCCESS (2.66s)
  File 9: SUCCESS (3.93s)
  File 10: SUCCESS (2.76s)
  File 11: SUCCESS (4.96s)
  File 12: SUCCESS (2.76s)
  File 13: SUCCESS (3.93s)
  File 14: SUCCESS (4.89s)
  File 15: SUCCESS (3.62s)
  File 16: SUCCESS (1.90s)
  File 17: SUCCESS (3.67s)
  File 18: SUCCESS (3.18s)
  File 19: SUCCESS (2.38s)
  File 20: SUCCESS (4.15s)
  File 21: SUCCESS (1.52s)
  File 22: SUCCESS (2.44s)
  File 23: SUCCESS (2.03s)
  File 24: SUCCESS (3.80s)
  File 25: SUCCESS (3.44s)
  File 26: SUCCESS (1.60s)
  File 27: SUCCESS (1.78s)
  File 28: SUCCESS (2.02s)
  File 29: SUCCESS (3.86s)
  File 30: SUCCESS (2.36s)
  File 31: SUCCESS (3.34s)
  File 32: SUCCESS (1.66s)
  File 33: SUCCESS (2.87s)
  File 34: SUCCES

### Statistical Analysis

In [23]:
# Check vector store file batch status
def check_batch_status(batch_id):
    if not batch_id:
        print("No batch ID to check")
        return
    
    print(f"Checking status of batch: {batch_id}")
    try:
        batch = client.vector_stores.file_batches.retrieve(
            vector_store_id=vector_store_id,
            batch_id=batch_id
        )
        
        print(f"Batch status: {batch.status}")
        print(f"File counts - Total: {batch.file_counts.total}, Completed: {batch.file_counts.completed}")
        print(f"In progress: {batch.file_counts.in_progress}, Failed: {batch.file_counts.failed}")
        
        return batch.status
    except Exception as e:
        print(f"Failed to check batch status: {e}")
        return None

if batch_result:
    batch_status = check_batch_status(batch_result.id)

Checking status of batch: vsfb_3626cf46a360475392876cd251c7d13e
Batch status: completed
File counts - Total: 50, Completed: 50
In progress: 0, Failed: 0


In [24]:
# List vector store files
def list_vector_store_files():
    print("Listing files in vector store...")
    try:
        files = client.vector_stores.files.list(vector_store_id=vector_store_id)
        print(f"Total files in vector store: {len(files.data)}")
        
        status_counts = {}
        for file in files.data:
            status = file.status
            status_counts[status] = status_counts.get(status, 0) + 1
        
        print("File status counts:")
        for status, count in status_counts.items():
            print(f"  {status}: {count}")
            
    except Exception as e:
        print(f"Failed to list vector store files: {e}")

list_vector_store_files()

Listing files in vector store...
Total files in vector store: 20
File status counts:
  completed: 20


In [25]:
# Performance Summary
print("\n" + "="*70)
print("PERFORMANCE SUMMARY")
print("="*70)

print("Test 1 - Individual Upload + Individual Vector Store Addition (50 files):")
print(f"  File upload: {50/individual_duration:.2f} files/sec")
print(f"  Vector store addition: {individual_vs_count/individual_vs_duration:.2f} files/sec")
print(f"  Total time: {individual_duration + individual_vs_duration:.2f}s")

print(f"\nTest 2 - Batch Upload + Vector Store Batch (50 files):")
print(f"  File upload: {batch_file_count/batch_upload_time:.2f} files/sec")
print(f"  Vector store batch creation: {batch_creation_time:.2f}s")
print(f"  Total time: {batch_upload_time + batch_creation_time:.2f}s")

print(f"\nTest 3 - Concurrent Upload (50 files, 30 workers):")
print(f"  Success rate: {concurrent_success_count}/50 files")
print(f"  Upload rate: {concurrent_success_count/concurrent_duration:.2f} files/sec")

print(f"\nTest 4 - High Concurrency Stress Test (50 files, 50 workers):")
print(f"  Success rate: {stress_success_count}/50 files")
print(f"  Upload rate: {stress_success_count/stress_duration:.2f} files/sec")

# Calculate theoretical vs actual rates
total_successful_files = individual_vs_count + batch_file_count + concurrent_success_count + stress_success_count
print(f"\nOverall Results:")
print(f"  Total files processed: {total_successful_files}/200")
print(f"  Success rate: {total_successful_files/200*100:.1f}%")



PERFORMANCE SUMMARY
Test 1 - Individual Upload + Individual Vector Store Addition (50 files):
  File upload: 0.60 files/sec
  Vector store addition: 2.11 files/sec
  Total time: 107.16s

Test 2 - Batch Upload + Vector Store Batch (50 files):
  File upload: 0.68 files/sec
  Vector store batch creation: 2.10s
  Total time: 75.55s

Test 3 - Concurrent Upload (50 files, 30 workers):
  Success rate: 50/50 files
  Upload rate: 9.40 files/sec

Test 4 - High Concurrency Stress Test (50 files, 50 workers):
  Success rate: 50/50 files
  Upload rate: 8.92 files/sec

Overall Results:
  Total files processed: 200/200
  Success rate: 100.0%


### Housekeeping

In [26]:
# Remove test files and vector store
def cleanup():
    print("Cleaning up...")
    
    # Clean up local files
    if temp_dir and os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"Deleted test files directory: {temp_dir}")
    
    # Clean up uploaded files from Azure OpenAI
    print("Deleting uploaded files from Azure OpenAI...")
    all_uploaded_files = []
    
    # Collect all file IDs from different tests
    if 'individual_file_ids' in globals():
        all_uploaded_files.extend(individual_file_ids)
    if 'concurrent_file_ids' in globals():
        all_uploaded_files.extend([fid for fid in concurrent_file_ids if fid])
    if 'stress_file_ids' in globals():
        all_uploaded_files.extend([fid for fid in stress_file_ids if fid])
    
    try:
        vs_files = client.vector_stores.files.list(vector_store_id=vector_store_id)
        batch_file_ids = [f.id for f in vs_files.data]
        all_uploaded_files.extend(batch_file_ids)
    except Exception as e:
        print(f"Could not retrieve vector store files for cleanup: {e}")
    
    # Remove duplicates
    unique_file_ids = list(set(all_uploaded_files))
    print(f"Found {len(unique_file_ids)} unique files to delete")
    
    # Delete each file
    deleted_count = 0
    for file_id in unique_file_ids:
        try:
            client.files.delete(file_id)
            deleted_count += 1
            print(f"  Deleted file: {file_id}")
        except Exception as e:
            print(f"  Failed to delete file {file_id}: {e}")
    
    print(f"Successfully deleted {deleted_count}/{len(unique_file_ids)} files from Azure OpenAI")
    
    # Delete the vector store
    try:
        client.vector_stores.delete(vector_store_id)
        print(f"Deleted vector store: {vector_store_id}")
    except Exception as e:
        print(f"Failed to delete vector store: {e}")

cleanup()

Cleaning up...
Deleted test files directory: c:\Users\lturakulov\Downloads\ZZZ_TEMP\ZZZ_CASES_WITH_RCA\Bosch_File_Uploads\test_files
Deleting uploaded files from Azure OpenAI...
Found 170 unique files to delete
  Deleted file: assistant-DQxdfivWJyhL1hDMUKRyi5
  Deleted file: assistant-UJfr7qM53829R28L25vVyE
  Deleted file: assistant-A84a1vAmEfpDQ5hFxTN1C6
  Deleted file: assistant-6mwWuH5TAnL2zWxd1f9Ckg
  Deleted file: assistant-Ro9eZvwU7BxeQcx4MQEcqD
  Deleted file: assistant-FoqX8wSVqvZChFFwyeEtFa
  Deleted file: assistant-Jd7jjT9DnMmQNj98G5pDTM
  Deleted file: assistant-Bi2Ridf38WkupBne1gbDe1
  Deleted file: assistant-6JVzUDuYeSXALb3NazMAdf
  Deleted file: assistant-4WS7BvcYtUUJY48ZUHUqF1
  Deleted file: assistant-1jDEQuseGqnYvadUFqLwM2
  Deleted file: assistant-1cEG83zsUmDuS4H7xnnAGm
  Deleted file: assistant-Kxo4rT75HHHcSSJAFymdkK
  Deleted file: assistant-T1jcYbEt5dCtrWDwkUacF5
  Deleted file: assistant-4zhMKZumJY9HANVt4xWpiN
  Deleted file: assistant-GSwctpRL9jscKk6GW3XroM
  Del